In [ ]:
######################################
# Get radar data from knmi webservice
######################################

In [32]:

import folium
#import psycopg2 as ps
from urllib import request
import io
from datetime import datetime
from PIL import Image
from folium import plugins
from scipy.ndimage import imread
from pyproj import Proj, transform

def wgs842Streetview(lat,lon):
    wgs84 = Proj("+init=EPSG:4326")
    streetview = Proj("+init=EPSG:3857")
    tlon,tlat = transform(wgs84,streetview,lon,lat)
    return tlat,tlon


def overlayRadarImage(latlonmax,latlonmin,w,h,dtg):

    url = "http://geoservices.knmi.nl/cgi-bin/RADNL_OPER_R___25PCPRR_L3.cgi?SERVICE=WMS&&SERVICE=WMS&VERSION="
    url = url + "1.3.0&REQUEST=GetMap&LAYERS=RADNL_OPER_R___25PCPRR_L3_COLOR&WIDTH=%s&HEIGHT=%s&CRS=EPSG%%3A3857"
    url = url + "&BBOX=%s,%s,%s,%s&STYLES=default&FORMAT"
    url = url + "=image/png&TRANSPARENT=TRUE&&time=%s"

    dtgstr = dtg.strftime('%Y-%m-%dT%H%%3A%M%%3A00Z')

    slatmin, slonmin = wgs842Streetview(latlonmin[0],latlonmin[1])
    slatmax,slonmax = wgs842Streetview(latlonmax[0],latlonmax[1])

    print(latlonmin)
    print(latlonmax)
    print(slatmin,slonmin)
    print(slatmax, slonmax)

    #URL = URL %(w,h,slatmin,slonmin,slatmax,slonmax,dtgstr)
    url = url % (w, h, slonmin, slatmin, slonmax, slatmax, dtgstr)
    print(url)

    file = io.BytesIO(request.urlopen(url).read())
    data = imread(file)

    location = [(latlonmax[0] + latlonmin[0])/2,(latlonmax[1] + latlonmin[1])/2]
    bounds=[latlonmax,latlonmin]
    m = folium.Map(location=location, zoom_start=7)
    m.add_child(plugins.ImageOverlay(data,bounds,opacity=0.8))
    return m

    #img = Image.open(file)

latlonmax = [53.5,3.0]
latlonmin = [50,8.0]
h=600
w=1200
dtg = datetime(2017,8,1,18,5)

overlayRadarImage(h=h,w=w, latlonmax=latlonmax,latlonmin=latlonmin,dtg=dtg)

#2017-08-10T15%3A00%3A00Z
#[50, 6.0]
#[53.5, 3.0]
#6446275.841017158 667916.9447596424
#7076025.276180581 333958.4723798198


[50, 8.0]
[53.5, 3.0]
6446275.841017158 890555.9263461898
7076025.276180581 333958.4723798198
http://geoservices.knmi.nl/cgi-bin/RADNL_OPER_R___25PCPRR_L3.cgi?SERVICE=WMS&&SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=RADNL_OPER_R___25PCPRR_L3_COLOR&WIDTH=1200&HEIGHT=600&CRS=EPSG%3A3857&BBOX=890555.9263461898,6446275.841017158,333958.4723798198,7076025.276180581&STYLES=default&FORMAT=image/png&TRANSPARENT=TRUE&&time=2017-08-01T18%3A05%3A00Z
